# Import

In [10]:
import requests
import pandas as pd
import os

# Initialize

In [12]:
def initialize():
    global name, name_id, data, player, played_rounds, played_Least_times, played_ave, played_total_times, result
    if not os.path.exists("RECORD.xlsx"):
        df = pd.DataFrame(columns=['Name', 'Rounds', 'Least_times', 'Average_times', 'Total_times'])
        df.to_excel("RECORD.xlsx")
    info = pd.read_excel("RECORD.xlsx")
    name = input("Please input your name: ")
    if name in list(info['Name']): # convert to a list to check !!!
        name_id = info[info['Name'] == name].index[0] # get the row index of the player's name.
        data = info.loc[name_id] # the data's shape looks like a form, not a list.
        player = data["Name"] # so we need to use this way to acquire specific info.
        played_rounds = data["Rounds"]
        played_Least_times = data["Least_times"]
        played_ave = data["Average_times"]
        played_total_times = data["Total_times"]
        print("Hi %s, you have played %d rounds, you guessed the answer in at least %d times, "\
                  "you guessed the answer on average %.2f times each round."
                  % (player, played_rounds, played_Least_times, played_ave))
#         print(info)
#         print(name_id)
    else:
        info.loc[''] = [name, 0, 0, 0.0, 0] # creat a new player at the rear of the sheet.
        info.to_excel("RECORD.xlsx", index=False) # remember to save it !!!
        info = pd.read_excel("RECORD.xlsx") # open it again, so that we get the new player's index.
#         print(info)
        name_id = info[info['Name'] == name].index[0]
#         print("name_id: ", name_id)
        played_rounds = 0 # remember to initialize these variables after we creat a new player
        played_Least_times = 0
        played_ave = 0.0 # pay attention: average should be a float !
        played_total_times = 0
        print("%s you have played 0 round, you guessed the answer in at least 0 time, "\
        "you guess the answer on average 0 time each round." % name)
        info.to_excel("RECORD.xlsx", index=False)
    print("\nRound1")
    result = get_result()
    count = 1
    Round = 1
    num = []
#     print(result)

# Guess and check

In [3]:
def guess_and_check():
    global guess_number, Bingo
    num_type = False
    while num_type == False: # ensure that players enter the right type.
        try:
            guess_number = int(input("Please enter your answer: "))
        except:    
            print("You must enter an integer!!!")
        else:
            if type(guess_number) == int:
                Bingo = False
                break                

# Compare

In [4]:
def compare(result, count):
    guess_and_check()
    if guess_number < result:
        print("Too small!")
        return False
    elif guess_number > result:
        print("Too big!")
        return False
    else:
        print("Bingo!!!")
        print("You guessed %d times" % count + '\n') 
        num.append(count) # won't count until player enter the right type and guess the answer.
        return replay()  # there must be "return replay()"!!! if omit "return", it doesn't work!!!    

# Replay
If the player want to replay, we update the answer. Otherwise, we estimate their performance and end the game.

In [5]:
def replay():
    global Round, count, result, new_least_times, new_rounds, new_total, new_ave
    if played_Least_times == 0:  # Pay attention: the new_least_times equals min(num) when played_least_times = 0!!!
        new_least_times = min(num)
    else:
        new_least_times = min(min(num), played_Least_times)
    
    restart = input("Do you want to replay? (Press 'Y' or 'y' to continue, press any other keys to quit): ")
    if restart == "Y" or restart == "y":
        result = get_result() # Update the result !!!
#         print(result)
        Round += 1
        print('\n' + "Round%d" % Round)
        count = 0
        return False
    else:
        average = sum(num) / len(num)
        new_rounds = Round + played_rounds
        new_total = played_total_times + sum(num)
        new_ave = new_total / new_rounds    # Update the variables !!!
        print('\n' + "THIS TIME: you played %d rounds, you guessed the answer in at least %d times, "\
              "you need to guess %.2f times per round"
              % (Round, min(num), average))           
        print('\n' + "TOTAL: you have played %d rounds, you guessed the answer in at least %d times, "\
              "you need to guess %.2f times per round" % (new_rounds, new_least_times, new_ave))
        update()
        return True  

# Update

In [6]:
def update():
    info2 = pd.read_excel("RECORD.xlsx")
    info2.loc[name_id] = [name, new_rounds, new_least_times, new_ave, new_total]  # update !!!
    info2.to_excel("RECORD.xlsx", index=False) # must save !!!                  

# Get result

In [7]:
def get_result():
    url = "https://python666.cn/cls/number/guess/"
    req = requests.get(url)
    req.encoding = "gbk"
    result = eval(req.text)
    return result           

# Play

In [8]:
def play():
    global result, count
    initialize()
    while not compare(result, count):
        count += 1

In [9]:

play()

Please input your name: nqpo
Hi nqpo, you have played 6 rounds, you guessed the answer in at least 1 times, you guessed the answer on average 5.67 times each round.

Round1
Please enter your answer: 50
Too small!
Please enter your answer: 75
Too big!
Please enter your answer: 62
Too big!
Please enter your answer: 56
Too small!
Please enter your answer: 59
Too big!
Please enter your answer: 58
Bingo!!!
You guessed 6 times

Do you want to replay? (Press 'Y' or 'y' to continue, press any other keys to quit): y

Round2
Please enter your answer: 50
Too small!
Please enter your answer: 50f
You must enter an integer!!!
Please enter your answer: ,.
You must enter an integer!!!
Please enter your answer:  
You must enter an integer!!!
Please enter your answer: [
You must enter an integer!!!
Please enter your answer: 75
Too big!
Please enter your answer: 58
Too small!
Please enter your answer: 63
Too small!
Please enter your answer: 60
Too small!
Please enter your answer: 70
Too small!
Please ent